In [3]:
import sys
print sys.executable

paths = ['', '/Users/dyawitz/anaconda/bin', '/Users/dyawitz/anaconda/lib/python27.zip', '/Users/dyawitz/anaconda/lib/python2.7/plat-darwin', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/dyawitz/anaconda/lib/python2.7/lib-tk', '/Users/dyawitz/anaconda/lib/python2.7/lib-old', '/Users/dyawitz/anaconda/lib/python2.7/lib-dynload', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/setuptools-18.5-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/aeosa', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/IPython/extensions', '/Users/dyawitz/.ipython']

for i in paths:
   sys.path.append(i)

/Users/dyawitz/anaconda/envs/py27/bin/python


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [5]:
weather_dict = joblib.load('weather_dict.pkl')

In [6]:
weather_dict

{'CAPITL': ('kalb', 'Capital', 'Albany'),
 'CENTRL': ('ksyr', 'Central', 'Syracuse'),
 'DUNWOD': ('klga', 'Dunwoodie', 'Yonkers'),
 'GENESE': ('kroc', 'Genese', 'Rochester'),
 'HUD VL': ('kpou', 'Hudson Valley', 'Poughkeepsie'),
 'LONGIL': ('kjfk', 'Long Island', 'NYC'),
 'MHK VL': ('krme', 'Mohawk Valley', 'Utica'),
 'MILLWD': ('klga', 'Millwood', 'Yonkers'),
 'N.Y.C.': ('kjfk', 'NYC', 'NYC'),
 'NORTH': ('kpbg', 'North', 'Plattsburgh'),
 'WEST': ('kbuf', 'West', 'Buffalo')}

### Import all the region data from 2012-2015


# 1. Format datetime columns

In [7]:
def format_datetime(weather, loads):
    #Format datetime columns:
    weather['date'] = weather.dateutc.apply(lambda x: pd.to_datetime(x).date())
    weather['timeest'] = weather.timeest.apply(lambda x: pd.to_datetime(x).time())
    foo = weather[['date', 'timeest']].astype(str)
    weather['timestamp'] = pd.to_datetime(foo['date'] + ' ' + foo['timeest'])
    loads['timestamp'] = loads.timestamp.apply(lambda x: pd.to_datetime(x))
    return weather, loads

# 2. Add weather data to loads data
Weather data is on hourly intervals, loads data is every five minutes. This is a function to merge based on the nearest datetime, using K-nearest neighbors

In [8]:
from sklearn.neighbors import NearestNeighbors

def find_nearest(group, match, groupname):
    nbrs = NearestNeighbors(1).fit(match['timestamp'].values[:, None])
    dist, ind = nbrs.kneighbors(group['timestamp'].values[:, None])

    group['nearesttime'] = match['timestamp'].values[ind.ravel()]
    return group

# 3. Create features

Create new features based on the timeseries. These first features come from [Barta et al. 2015](http://arxiv.org/pdf/1506.06972.pdf), who applied probabalistic modeling techniques (such as Gradient-Boosting Regression) to forecast electricity prices:

    `dow`: day of the week (integer 0-6)
    `doy`: day of the year (integer 0-365)
    `day`: day of the month (integer 1-31)
    `woy`: week of the year (integer 1-52)
    `month`: month of the year (integer 1-12)
    `hour`: hour of the day (integer 0-23)
    `minute`: minute of the day (integer 0-1339)
    
    `t_m24`: load value from 24 hours earlier
    `t_m48`: load value from 48 hours earlier
    `tdif`: difference between load and t_m24


In [9]:
#datetime value of one day
pday = pd.Timedelta('1 day')

def get_prev_days(x, n_days):
    '''Take a datetime (x) in the 'full' dataframe, and outputs the load value n_days before that datetime'''
    try:
        lo = full[full.timestamp == x - n_days*pday].load.values[0]
    except:
        lo = full[full.timestamp == x].load.values[0]
    return lo 

In [10]:
def add_time_features(df):
    full = df.copy()
    full['dow'] = full.timestamp.apply(lambda x: x.dayofweek)
    full['doy'] = full.timestamp.apply(lambda x: x.dayofyear)
    full['day'] = full.timestamp.apply(lambda x: x.day)
    full['month'] = full.timestamp.apply(lambda x: x.month)
    full['year'] = full.timestamp.apply(lambda x: x.year)
    full['hour'] = full.timestamp.apply(lambda x: x.hour)
    full['minute'] = full.timestamp.apply(lambda x: x.hour*60 + x.minute)

    full['t_m24'] = full.timestamp.apply(get_prev_days, args=(1,))
    full['t_m48'] = full.timestamp.apply(get_prev_days, args=(2,))
    full['tdif'] = full['load'] - full['t_m24']
    return full

# Run this for every subset of NYS data

In [13]:
k = weather_dict.keys()

In [15]:
for region in k:
    
    place = weather_dict[region][1].lower().replace(' ','')
    airport = weather_dict[region][0]

    #load in the data
    loads = pd.read_csv('../data/nyiso/all/{0}.csv'.format(place))
    weather = pd.read_csv('../data/wunderground/{0}_all.csv'.format(airport))

    #remove loose headers
    weather = weather[weather.winddirection != 'winddirection']
    
    #format datetime columns
    weather, loads = format_datetime(weather, loads)

    #combine using KNN
    loads = find_nearest(loads,weather,'timestamp')
    full = loads.merge(weather, left_on='nearesttime', right_on='timestamp')

    #Remove and rename redundant columns 
    full = full[['timestamp_x', 'load', 'nearesttime', 'temperaturef', \
                'dewpointf', 'humidity', 'sealevelpressurein', 'winddirection', 'windspeedmph', \
                'precipitationin']].rename(columns={'timestamp_x': 'timestamp', 'nearesttime':'weathertime'})

    #Create features
    full = add_time_features(full)

    #Export to csv
    full.to_csv('full_{0}_features.csv'.format(place), index=False)

In [17]:
#datetime value of one day
phour = pd.Timedelta('1 hour')

def get_prev_hours(x, n_hours):
    '''Take a datetime (x) in the 'full' dataframe, and outputs the load value n_days before that datetime'''
    try:
        lo = full[full.timestamp == x - n_hours*phour].load.values[0]
    except:
        lo = full[full.timestamp == x].load.values[0]
    return lo 

In [18]:
for region in k:
    place = weather_dict[region][1].lower().replace(' ','')
    airport = weather_dict[region][0]
    
    full = pd.read_csv('full_{0}_features.csv'.format(place))
    
    full['t_m1'] = full.timestamp.apply(get_prev_hours, args=(1,))
    
    full.to_csv('full_{0}_features.csv'.format(place), index=False)
    
    print "%s done" % place

KeyboardInterrupt: 

In [19]:
place

'longisland'

In [20]:
k

['LONGIL', 'WEST', 'N.Y.C.']

In [21]:
full

,timestamp,load,weathertime,temperaturef,dewpointf,humidity,sealevelpressurein,winddirection,windspeedmph,precipitationin,dow,doy,day,month,year,hour,minute,t_m24,t_m48,tdif
0,2012-01-01 00:00:00,2245.1,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,0,2245.1,2245.1,0.0
1,2012-01-01 00:00:00,2245.1,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,0,2245.1,2245.1,0.0
2,2012-01-01 00:05:00,2228.7,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,5,2228.7,2228.7,0.0
3,2012-01-01 00:05:00,2228.7,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,5,2228.7,2228.7,0.0
4,2012-01-01 00:10:00,2211.4,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,10,2211.4,2211.4,0.0
5,2012-01-01 00:10:00,2211.4,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,10,2211.4,2211.4,0.0
6,2012-01-01 00:15:00,2205.0,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,15,2205.0,2205.0,0.0
7,2012-01-01 00:15:00,2205.0,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,15,2205.0,2205.0,0.0
8,2012-01-01 00:20:00,2203.5,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,20,2203.5,2203.5,0.0
9,2012-01-01 00:20:00,2203.5,2012-01-01 00:51:00,46.0,37.0,71,30.04,WNW,10.4,0,6,1,1,1,2012,0,20,2203.5,2203.5,0.0
